In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import math

Approximate Median Significance (AMS) defined as: $AMS = \sqrt{2 { (s + b + b_r) log[1 + (s/(b+b_{reg}))] - s}}$    
        
where 

- $b_{reg} = 10$ is a regulization term (set by the contest),
- $b = \sum_{i=1}^{n} w_i, y_i=0$ is sum of weighted background (incorrectly classified as signal),
- $s = \sum_{i=1}^{n} w_i, y_i=1$ is sum of weighted signals (correctly classified as signal),
- $log$ is natural logarithm

In [ ]:

def calcAMS(s,b):    
    br = 10.0
    radicand = 2 *( (s+b+br) * math.log (1.0 + s/(b+br)) -s)
    if radicand < 0:
        print('radicand is negative. Exiting')
        exit()
    else:
        ams = math.sqrt(radicand)
        print("AMS:", ams)
        return ams

Following this definition, we can derive a maximum AMS by simply summing the weights of all positive labels.

In [ ]:

def calcWeightSums(weights,preds,labels):
    s = 0
    b = 0
    for j in list(range(0,len(preds))):
        pred = preds[j]
        label = labels[j]
        weight = weights[j]
        if pred > 0.:
            if label > 0.:
                s += weight
            else:
                b += weight
    return s,b

In [ ]:

def calcMaxAMS(weights,labels):
    s,b = calcWeightSums(weights,labels,labels)
    ams = calcAMS(s,b)
    print("Found", int(labels.cumsum()[-1]), "signals.")
    print("Weightsums signal:", s, "| background:", b)
    print("Maximum AMS possible with this Data:", ams)
    return ams

We generate AMS with good seperable toy-data, starting with the maximum AMS. 
The data of the actual challenge is weighted to punish wrong-identified signals significantly harder than wrong background. Our toy-data will do so by using its signal-probability as weight, the features are randomized by normal distributions.


In [ ]:

def generateFeature(label, mu_s, mu_b, sigma_s=5, sigma_b=5):
    if label is 1:
        mu = mu_s
        sigma = sigma_s
    else:
        mu = mu_b
        sigma = sigma_b
    return np.random.normal(mu,sigma)

In [ ]:

def createToyData(n = 100,dim = 3,s_prob = 0.05):
    data= np.zeros(shape = (n,dim),dtype=float)
    if dim < 3:
        print("Operation canceled.",
              "Data should have at least one",
              "additional dimension besides weights and labels.",
              "(dim >=3)")
        return None
    data[:,0] = np.random.rand(n) #weights
    for i in range(0,n):
        if data[i,0] <= s_prob: # label-determination
            label = 1
        else:
            label = 0
        data[i,1] = label
        for j in range(2,dim):
            #mu_s=j*5
            #mu_b=j*20
            data[i,j]=generateFeature(label,mu_s=(j-1)*5,mu_b=(j-1)*20)
    return data

In [ ]:

n = 100000
prob = 0.05
data = createToyData(n,dim=10,s_prob=prob)

In [ ]:

weights = data[:,0]
labels = data[:,1]
calcMaxAMS(weights,labels);

We randomly guess labels for a solution for a second AMS with knowledge about the toydatas signal-probability.

In [ ]:

sol_weights = np.random.rand(n)
sol = np.zeros(n)
for i in range(0,n):
    if sol_weights[i] <= prob: # label-determination
        sol[i] = 1
    else:
        sol[i] = 0

In [ ]:

s,b = calcWeightSums(weights,sol,labels)
calcAMS(s,b);